In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

'''
@author : wangkechn
https://github.com/wangkechn
'''
#载入数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)


In [2]:
#输入图片是28*28
n_imputs = 28 #输入一行 一行有28个数据
max_time = 28 #一共28行
lstm_size = 100 #隐层单元
n_classes = 10 #10个分类
batch_size = 50 #每个批次50个样本
n_batch = mnist.train.num_examples // batch_size #计算一共有多少个批次

#这里none表示第一个维度可以是任意长度
x = tf.placeholder(tf.float32,[None,784])#28*28
#正确的标签
y = tf.placeholder(tf.float32,[None,10])

#初始化权值 偏置值
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
biases = tf.Variable(tf.constant(0.1,shape = [n_classes]))

#定义RNN网络

def RNN(X, weights, biases):
    #inputs = [batch_size, max_time, n_inputs]
    inputs = tf.reshape(X,[-1, max_time, n_imputs])
    #定义LSTM基本CELl
    #lstm_cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_size)
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    #final_state[state,batch_size(50),cell.state_size(lstm_size100)]
    #final_state[0] 是cell state  中间的信号
    #final_state[1]是hidden state
    #outputs: The RNN output `Tensor`.
    #If time_major == False (default), this will be a `Tensor` shaped:
    # `[batch_size, max_time, cell.output_size(100)]`.

    #If time_major == True, this will be a `Tensor` shaped:
    #  `[max_time, batch_size, cell.output_size]`.
    #outputs记录了共max_time的输出 最后一一个
    #final_state[1]只记录了最后一次
    
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype = tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)
    return results

#计算RNN的返回结果
prediction = RNN(x,weights,biases)

#交叉熵代价函数   该版本的之后会被移除 详见下面的WARNING
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用adamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#初始化变量
init = tf.global_variables_initializer()


#equal 比较相同 返回True  结果存放在布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  #argmax()返回一维向量中最大的值的索引位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #cast()转化格式 True为1.0 False:0.0 变为float32类型

with tf.Session() as sess:
    sess.run(init)
    #重新运行程序 否则图会重叠
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
                         
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter' + str(epoch) + ",Testing Accuracy "+ str(acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter0,Testing Accuracy 0.6856
Iter1,Testing Accuracy 0.8095
Iter2,Testing Accuracy 0.8956
Iter3,Testing Accuracy 0.9105
Iter4,Testing Accuracy 0.9263
Iter5,Testing Accuracy 0.9374
